In [1]:
!pip install kaggle
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download gti-upm/leapgestrecog

Dataset URL: https://www.kaggle.com/datasets/gti-upm/leapgestrecog
License(s): CC-BY-NC-SA-4.0
100% 2.12G/2.13G [00:57<00:00, 40.2MB/s]
100% 2.13G/2.13G [00:57<00:00, 39.7MB/s]


In [3]:
!unzip /content/leapgestrecog.zip

Streaming output truncated to the last 5000 lines.
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0001.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0002.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0003.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0004.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0005.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0006.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0007.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0008.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0009.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0010.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0011.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0012.png  
  inflating: leapgestrecog/leapGestRe

In [4]:
import numpy as np
import pandas as pd
import os
import cv2

import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import matplotlib.pyplot as plt

In [6]:
dir = '/content/leapGestRecog'

In [7]:
images = []
labels = []
for directory in os.listdir(dir):
  for subDir in os.listdir(os.path.join(dir,directory)):
    for img in os.listdir(os.path.join(dir, directory, subDir)):
      img_path = os.path.join(dir, directory, subDir, img)
      images.append(img_path)
      labels.append(subDir)

In [8]:
Iseries = pd.Series(images, name="Images")
Lseries = pd.Series(labels, name="labels")
hand_gesture_data = pd.concat([Iseries, Lseries], axis=1)
hand_gesture_df = pd.DataFrame(hand_gesture_data)

In [9]:
print(hand_gesture_df.head())

                                              Images   labels
0  /content/leapGestRecog/06/10_down/frame_06_10_...  10_down
1  /content/leapGestRecog/06/10_down/frame_06_10_...  10_down
2  /content/leapGestRecog/06/10_down/frame_06_10_...  10_down
3  /content/leapGestRecog/06/10_down/frame_06_10_...  10_down
4  /content/leapGestRecog/06/10_down/frame_06_10_...  10_down


In [10]:
pd.Series(labels).value_counts()

10_down          2000
03_fist          2000
09_c             2000
06_index         2000
04_fist_moved    2000
08_palm_moved    2000
07_ok            2000
01_palm          2000
02_l             2000
05_thumb         2000
Name: count, dtype: int64

In [11]:
X_train, X_test = train_test_split(hand_gesture_df, test_size=0.2, random_state=42)
train_set, val_set = train_test_split(hand_gesture_df, test_size=0.3, random_state=42)

In [13]:
image_gen = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet_v2.preprocess_input)
train = image_gen.flow_from_dataframe(dataframe= train_set,x_col="Images",y_col="labels",
                                      target_size=(244,244),
                                      color_mode='rgb',
                                      class_mode="categorical",
                                      batch_size=4,
                                      shuffle=False
                                     )
test = image_gen.flow_from_dataframe(dataframe= X_test,x_col="Images", y_col="labels",
                                     target_size=(244,244),
                                     color_mode='rgb',
                                     class_mode="categorical",
                                     batch_size=4,
                                     shuffle= False
                                    )
val = image_gen.flow_from_dataframe(dataframe= val_set,x_col="Images", y_col="labels",
                                    target_size=(244,244),
                                    color_mode= 'rgb',
                                    class_mode="categorical",
                                    batch_size=4,
                                    shuffle=False
                                   )

Found 14000 validated image filenames belonging to 10 classes.
Found 4000 validated image filenames belonging to 10 classes.
Found 6000 validated image filenames belonging to 10 classes.


In [14]:
classes=list(train.class_indices.keys())
print (classes)

['01_palm', '02_l', '03_fist', '04_fist_moved', '05_thumb', '06_index', '07_ok', '08_palm_moved', '09_c', '10_down']


In [20]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=(8, 8), strides=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),

    keras.layers.MaxPool2D(pool_size=(2, 2)),

    keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),

    keras.layers.MaxPool2D(pool_size=(2, 2)),

    keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),

    keras.layers.MaxPool2D(pool_size=(2, 2)),

    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.SGD(learning_rate=0.001),
    metrics=['accuracy']
)

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 73, 73, 128)       24704     
                                                                 
 batch_normalization_15 (Ba  (None, 73, 73, 128)       512       
 tchNormalization)                                               
                                                                 
 conv2d_16 (Conv2D)          (None, 73, 73, 256)       295168    
                                                                 
 batch_normalization_16 (Ba  (None, 73, 73, 256)       1024      
 tchNormalization)                                               
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 36, 36, 256)       0         
 ng2D)                                                           
                                                      

In [21]:
history = model.fit(train, epochs=, validation_data=val)

Epoch 1/10
3500/3500 [==============================] - 153s 42ms/step - loss: 0.3153 - accuracy: 0.9357 - val_loss: 0.0069 - val_accuracy: 0.9975
Epoch 2/10
3500/3500 [==============================] - 149s 43ms/step - loss: 0.0056 - accuracy: 0.9989 - val_loss: 0.0019 - val_accuracy: 0.9993
Epoch 3/10
3500/3500 [==============================] - 148s 42ms/step - loss: 0.0028 - accuracy: 0.9996 - val_loss: 0.0028 - val_accuracy: 0.9987
Epoch 4/10
3500/3500 [==============================] - 149s 43ms/step - loss: 0.0026 - accuracy: 0.9996 - val_loss: 0.0013 - val_accuracy: 0.9997
Epoch 5/10
3500/3500 [==============================] - 149s 42ms/step - loss: 0.0018 - accuracy: 0.9996 - val_loss: 0.0015 - val_accuracy: 0.9997
Epoch 6/10
3500/3500 [==============================] - 149s 43ms/step - loss: 9.4004e-04 - accuracy: 0.9999 - val_loss: 0.0013 - val_accuracy: 0.9997
Epoch 7/10
3500/3500 [==============================] - 149s 43ms/step - loss: 7.3695e-04 - accuracy: 0.9999 - val

In [24]:
model.evaluate(test)

1000/1000 [==============================] - 16s 16ms/step - loss: 0.0019 - accuracy: 0.9995


[0.0019300212152302265, 0.9994999766349792]

In [28]:
pred = model.predict(test)
pred = np.argmax(pred, axis=1)
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred2 = [labels[k] for k in pred]
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

y_test = X_test.labels
print(classification_report(y_test, pred2))
print("Accuracy of the Model:","{:.1f}%".format(accuracy_score(y_test, pred2)*100))

1000/1000 [==============================] - 16s 16ms/step
               precision    recall  f1-score   support

      01_palm       1.00      1.00      1.00       409
         02_l       1.00      1.00      1.00       410
      03_fist       1.00      1.00      1.00       404
04_fist_moved       1.00      1.00      1.00       377
     05_thumb       1.00      1.00      1.00       366
     06_index       1.00      1.00      1.00       418
        07_ok       1.00      1.00      1.00       403
08_palm_moved       1.00      1.00      1.00       392
         09_c       1.00      1.00      1.00       404
      10_down       1.00      1.00      1.00       417

     accuracy                           1.00      4000
    macro avg       1.00      1.00      1.00      4000
 weighted avg       1.00      1.00      1.00      4000

Accuracy of the Model: 100.0%
